In [1]:
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()

/Users/user/PycharmProjects/pythonProject1/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Get current spot rates for top ten by marketcap

# Parameters for the API request
parameters = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 10,
    'page': 1,
    'sparkline': False,
    'locale': 'en'
}

# Fetching market data from CoinGecko API
coin_market_data = cg.get_coins_markets(**parameters)

# Initialize a dictionary to store the current spot rate data
current_spot_rate_dict = {}

# Extracting current spot rate data and storing in the dictionary
for coin in coin_market_data:
    coin_id = coin['id']
    current_price = coin['current_price']
    current_spot_rate_dict[coin_id] = current_price

# Display the dictionary
print(current_spot_rate_dict)

{'bitcoin': 42874, 'ethereum': 2566.79, 'tether': 0.999846, 'binancecoin': 302.34, 'solana': 96.58, 'ripple': 0.575353, 'usd-coin': 1.0, 'staked-ether': 2579.03, 'cardano': 0.548974, 'avalanche-2': 36.34}


In [3]:
# Provided data
data = coin_market_data

# Extracting 'id' values
cryptocurrency_ids = [item['id'] for item in data]

# Printing extracted 'id' values
print(cryptocurrency_ids)

['bitcoin', 'ethereum', 'tether', 'binancecoin', 'solana', 'ripple', 'usd-coin', 'staked-ether', 'cardano', 'avalanche-2']


In [4]:
# Get historical spot rates from month ago
import time
from pycoingecko import CoinGeckoAPI

# Initialize CoinGeckoAPI
cg = CoinGeckoAPI()

# List of cryptocurrency IDs you want to iterate through
cryptocurrency_ids = ['bitcoin', 'ethereum', 'tether', 'binancecoin', 'solana', 'ripple', 'usd-coin', 'staked-ether', 'cardano', 'avalanche-2']

# Dictionary to hold historical spot rate data
month_spot_rate_data = {}

# Loop through each cryptocurrency ID
for crypto_id in cryptocurrency_ids:
    try:
        # Fetch historical data for each cryptocurrency
        base_data = cg.get_coin_history_by_id(id=crypto_id, vs_currency='usd', date='12-12-2023', localization='false', community='false')

        # Extract the spot rate for each cryptocurrency
        base_spot_rate = base_data['market_data']['current_price']['usd']

        # Add the spot rate data to the dictionary
        month_spot_rate_data[crypto_id] = base_spot_rate

    except Exception as e:
        print(f"Error fetching data for {crypto_id}: {e}")

    # Sleep to avoid hitting rate limits
    time.sleep(3)

# Print the spot rate data dictionary
print("Spot Rate Data on 12-12-2023:")
for crypto_id, spot_rate in month_spot_rate_data.items():
    print(f"{crypto_id}: {spot_rate}")


Spot Rate Data on 12-12-2023:
bitcoin: 41200.96272610168
ethereum: 2222.3468384000316
tether: 0.9995940304535856
binancecoin: 246.2875566773758
solana: 70.20241559474354
ripple: 0.6195589019990316
usd-coin: 1.0008410353273296
staked-ether: 2221.1298351226765
cardano: 0.5503522027851744
avalanche-2: 38.200052884551354


In [5]:
def calculate_crypto_index(base_level, weights, spot_rates_current, spot_rates_rebalance, divisor):
    """
    Calculate the cryptocurrency index level.

    :param base_level: The index level at the base date (float).
    :param weights: List of weights for each digital asset (list of floats).
    :param spot_rates_current: List of spot rates at the current time (list of floats).
    :param spot_rates_rebalance: List of spot rates at the rebalance time (list of floats).
    :param divisor: The divisor at the current time (float).
    :return: The calculated index level (float).
    """
    if not (len(weights) == len(spot_rates_current) == len(spot_rates_rebalance)):
        raise ValueError("Length of weights, spot rates at current time, and spot rates at rebalance time must be equal.")

    # Calculate the sum of weighted rate changes
    weighted_sum = sum(w * (current / rebalance) for w, current, rebalance in zip(weights, spot_rates_current, spot_rates_rebalance))

    # Calculate the index level
    index_level = base_level * weighted_sum / divisor

    return index_level

# Example usage:
base_level = 1000  # Example base level
weights = [0.1]*10  # Example weights for ten digital assets
spot_rates_current = list(current_spot_rate_dict.values())  # Spot rates at current time
spot_rates_rebalance = list(month_spot_rate_data.values())  # Spot rates at rebalance time
divisor = 10  # Example divisor

index_level = calculate_crypto_index(base_level, weights, spot_rates_current, spot_rates_rebalance, divisor)
print(f"The cryptocurrency index level is: {index_level}")

The cryptocurrency index level is: 108.36921830966207
